# Entrega 1, Grupo 02 - Arboles de decisión

- Santiago Alaniz, 5082647-6, santiago.alaniz@fing.edu.uy
- Bruno De Simone, 4914555-0, bruno.de.simone@fing.edu.uy
- María Usuca, 4891124-3, maria.usuca@fing.edu.uy



## Objetivo

Implementar una variante de `ID3` para construir un modelo que explique la deserción de estudiantes en la universidad.
 
Se pide:

- **(a)** Implementar una variante del algoritmo `ID3` agregandole los siguientes *hiperparametros*:
    - **i)** `min_samples_split`: cantidad mínima de ejemplos para generar un nuevo nodo; en caso de que no se llegue a la cantidad requerida, se debe formar una hoja.
    - **ii)** `min_split_gain`: ganancia mínima requerida para partir por un atributo; si ningún atributo llega a ese valor, se debe formar una hoja.
- **(b)** Utilizar el algoritmo implementado en **(a)** para construir un arbol de decision, evaluar resultados utilizando el dataset provisto.
- **(c)** Discuta como afecta la variacion de los hiperparametros con los modelos obtenidos.
- **(d)** Corra los algoritmos de `scikit-learn` DecisionTreeClassifier, RandomForestClassifer y compare los resultados.

El dataset que vamos a considerar (con su debido preprocesamiento) es *Predict students dropout and accademic success* con **36 atributos y mas de 4000 instancias.**

## Diseño

El apartado de diseño engloba todas las decisiones que tomamos a la hora de cumplir con las subtareas planteadas en la seccion anterior. 

Podemos identificar las siguientes etapas:

- **Carga de datos y Particionamiento**: Inicialización de los datos de los archivos CSV en un DataFrame de Pandas.
- **Pre-procesamiento de datos**: Transformaciones necesarias para que los datos puedan ser utilizados por el modelo.
- **Algoritmo**: Comentarios sobre la implementación del algoritmo asi como las decisiones tomadas para su implementación.

### Carga de datos y particionamiento

En este apartado vamos a inicializar los datos siguiendo un esquema clásico de aprendizaje automático:

- **Carga de datos**: Cargamos los datos desde el fichero `csv` y los almacenamos en un `DataFrame` de `pandas`.
- **Particionamiento**: Particionamos los datos en dos conjuntos con `train_test_split` de `sklearn`.



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

CSV_PATH = './assets/data.csv'
SEED_NUMBER = 42069
TRAIN_SIZE = 0.8
TEST_SIZE = 0.2

data = pd.read_csv(CSV_PATH, sep=';')
train, test = train_test_split(data, test_size= TEST_SIZE, train_size= TRAIN_SIZE, random_state= SEED_NUMBER)

print(f'[ data: {data.shape[0]}, train: {train.shape[0]}, test: {test.shape[0]} ]')
train.head()

### Pre-procesamiento de los datos

Para el pre-procesado de los datos tomaremos en cuenta los siguientes puntos:

- Redefinición de los valores del atributo objetivo `Target` para que sean 0 y 1.
- Discretización de todos los atributos continuos.
- Comentario sobre el resto de los valores (discretos)

#### Redefinicion de los valores del atributo objetivo `Target`.

El atributo objetivo `Target` es un atributo categórico que indica el desenlace del estudiante en su vida académica. Este atributo tiene 3 posibles valores: 

- `Enrolled` (inscripto)
- `Dropout` (abandono)
- `Graduate` (graduado).

La idea es construir un modelo sobre la diserción de los estudiantes, por lo que se decide agrupar los valores `Enrolled` y `Graduate` en un solo valor. 

-  0 &rarr; `Dropout`
-  1 &rarr; `Enrolled` o `Graduate`

**Nota**: 
La siguiente redefinición de atributos genera un desbalance en el atributo `Target`. De todas formas, continuaremos con el análisis.

In [ ]:
for df in [train, test]:
    df['Target'] = df['Target'].apply(lambda x: 0 if x == 'Dropout' else 1)

train['Target'].value_counts()

#### Pre-procesamiento de atributos continuos.

La [discretizacion](https://en.wikipedia.org/wiki/Data_binning) provee un mecanismo para particionar valores continuos en un numero finito de valores discretos.

De los [36 atributos presentes](https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success) en el dataset, estos son listados como continuos:

- `Previous qualification (grade)`
- `Admission grade`
- `Unemployment rate`
- `Inflation rate`
- `GDP`

Para discretizar estos atributos, utlizaremos el modulo `scikit-learn.preprocessing`. 

En particular, la clase [`KBinsDiscretizer`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.KBinsDiscretizer.html#sklearn.preprocessing.KBinsDiscretizer) con los siguientes parametros no predeterminados:

- `encode= 'ordinal'` (codificacion de los bins) devuelve un array de enteros indicando a que bin pertenece cada valor.
- `strategy='kmeans'` (estrategia de discretizacion) utiliza el algoritmo de [k-means](https://en.wikipedia.org/wiki/K-means_clustering) para determinar los bins. 

Finalmente, identificar estos atributos en el dataset es una tarea sencilla, ya que son los unicos del tipo `float64`.

***Nota***: 

Hay un error en la [documentación de los datos](https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success), figuran como discretos dos campos representados con `float64`:

- `Curricular units 1st sem (grade)`
- `Curricular units 2nd sem (grade)` 

Decidimos discretizarlos de todas formas, ya que algunas de las entradas tienen valores no enteros.

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

float64_cols = data.select_dtypes(include=['float64']).columns

for float64_col in float64_cols:
    float64_col_discretizer = KBinsDiscretizer(subsample=None, encode='ordinal', strategy='kmeans')
    train[[float64_col]] = float64_col_discretizer.fit_transform(train[[float64_col]]).astype(int)
    test[[float64_col]] = float64_col_discretizer.transform(test[[float64_col]]).astype(int)

train[float64_cols].head()

## Algoritmo

El algoritmo a desarrollar es `ID3` como se presento en el teórico, con la incorporacion de ciertos meta-parametros que buscan evitar el sobreajuste del modelo.

Para lograr este objetivo, se tuvo en consideracion las siguientes sub-tareas:

1. Definir y obtener las variables necesarias para implementar `ID3` (Mitchell, 97, p56).
2. `ID3_utils.py`: Un modulo con estructuras/funciones auxiliares para la implementacion de `ID3`.
3. Codigo del algoritmo `ID3`.
3. `G02DecisionTrees.ID3Classifier`: Un diseño modular del algoritmo inspirado en `sklearn`.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from src.G02_algorithm import CustomID3Classifier
import matplotlib.pyplot as plt

X_train, y_train = train.drop(columns=['Target']), train['Target']

custom_clf = CustomID3Classifier(MIN_SAMPLES_SPLIT=0, MIN_SPLIT_GAIN=0)
custom_clf.fit(X_train, y_train)

In [ ]:
X_test, y_test = test.drop(columns=['Target']), test['Target']
predictions = custom_clf.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

conf_matrix = confusion_matrix(y_test, predictions)
disp = ConfusionMatrixDisplay(conf_matrix)
disp.plot(cmap=plt.cm.Blues, values_format='.2f')

plt.title("Confusion Matrix")
plt.show()



## Evaluación
- Qué conjunto de métricas se utilizan para la evaluación de la solución y su definición
- Sobre qué conjunto(s) se realiza el entrenamiento, ajuste de la solución, evaluación, etc. Explicar cómo se construyen estos conjuntos.

## Experimentación

- Presentar los distintos experimentos que se realizan y los resultados que se obtienen.

- La información de los resultados se presenta en tablas y en gráficos, de acuerdo a su naturaleza. Por ejemplo:

_En la gráfica 1, se observa el error cuadrático total del conjunto de entrenamiento a medida que pasan los juegos para el oponente X_

Preparación para ejecutar DecisionTreeClassifier y RandomForest

In [ ]:
MIN_SAMPLES_SPLIT = 10 #default=2
MAX_LEAF_NODES = 25
X_train, y_train = train.drop(columns=['Target']), train['Target']
X_test, y_test = test.drop(columns=['Target']), test['Target']

#### Ejecución de DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

decision_tree_clf = DecisionTreeClassifier(min_samples_split = MIN_SAMPLES_SPLIT, \
                                           max_leaf_nodes = MAX_LEAF_NODES)
decision_tree_clf.fit(X_train, y_train)
y_pred_decision_tree = decision_tree_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_decision_tree)
print("Precisión del modelo DT:", accuracy)

#### Ejecución de Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest_clf = RandomForestClassifier(min_samples_split = MIN_SAMPLES_SPLIT, \
                                        max_leaf_nodes = MAX_LEAF_NODES)
random_forest_clf.fit(X_train, y_train)
y_pred_random_forest = random_forest_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_random_forest)
print("Precisión del modelo RF:", accuracy)

#### Comparación de salidas

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Calcular matrices de confusión
cm_decision_tree = confusion_matrix(y_test, y_pred_decision_tree)
cm_random_forest = confusion_matrix(y_test, y_pred_random_forest)

## Crear objetos ConfusionMatrixDisplay
cmd_decision_tree = ConfusionMatrixDisplay(confusion_matrix=cm_decision_tree, display_labels=decision_tree_clf.classes_)
cmd_random_forest = ConfusionMatrixDisplay(confusion_matrix=cm_random_forest, display_labels=random_forest_clf.classes_)

# Mostrar las matrices de confusión
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

cmd_decision_tree.plot(cmap=plt.cm.Blues, ax=axes[0])
axes[0].set_title("Matriz de Confusión - Decision Tree")

cmd_random_forest.plot(cmap=plt.cm.Blues, ax=axes[1])
axes[1].set_title("Matriz de Confusión - Random Forest")

plt.tight_layout()
plt.show()

## Conclusión

Una breve conclusión del trabajo realizado. Por ejemplo: 
- ¿cuándo se dieron los mejores resultados del jugador?
- ¿encuentra alguna relación con los parámetros / oponentes/ atributos elegidos?
- ¿cómo mejoraría los resultados?